In [5]:
import httpx

# Define the request payload
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "Be helpulf"},
        {"role": "user", "content": "Hello, how are you?"}
    ],
    "temperature": 0.7,
    "stream": True  # Include this if your backend requires it explicitly
}

# Query parameters
params = {
    "user_id": "user-123",
    "session_id": "session-456",
    "channel_id": "channel-789",
    "channel_type": "web",
    "api_provider": "openai",
    "device_info": "ZGV2aWNlPXdlYjtzb3VyY2U9YnJvd3Nlcg=="
}

# Bearer token
bearer_token = "postgres"

# Stream the response
with httpx.stream(
    "POST",
    "http://127.0.0.1:5008/chat/completions",
    json=payload,
    params=params,
    headers={"Authorization": f"Bearer {bearer_token}"},
    timeout=None  # Disable timeout for long streaming responses
) as response:
    print("Status code:", response.status_code)
    print("Streaming response:")
    for line in response.iter_lines():
        if line:
            print(line)
            #print(line.decode("utf-8"))


In [6]:
#! touch /Users/sirsh/Downloads/example.txt && echo "hello world" >> /Users/sirsh/Downloads/example.txt


In [6]:
import httpx

# File to upload
file_path = "/Users/sirsh/Downloads/example.txt"
task_id = "task-abc-123"
bearer_token = "postgres"

# Open the file and prepare the multipart/form-data
with open(file_path, "rb") as f:
    files = {
        "file": ("example.txt", open(file_path, "rb"), "text/plain")
    }
    data = {
        "task_id": task_id
    }
    
    response = httpx.post(
        "http://127.0.0.1:5008/admin/content/upload",
        files=files,
        data=data,
        headers={
            "Authorization": f"Bearer {bearer_token}"
        },
        timeout=30.0
    )

# Print the response
print("Status code:", response.status_code)
print("Response body:", response.text)


In [15]:
from percolate.services.S3Service import S3Service
s3_service = S3Service()

result = s3_service.upload_file(
            project_name='test',
            file_name='test.txt',
            file_content=tmp_file,
            content_type=file.content_type
        )

In [1]:
from pathlib import Path
from percolate.services.S3Service import S3Service

file_path = Path("/Users/sirsh/Downloads/example.txt")
s3_service = S3Service()

with open(file_path,'rb') as f:
    result = s3_service.upload_file(
        project_name="test",
        file_name=file_path.name,
        file_content=f,  # send bytes
        content_type="text/plain"
    )

print("Upload result:", result)